In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.7 MB 7.6 MB/s 
     |████████████████████████████████| 101 kB 11.8 MB/s 
     |████████████████████████████████| 6.6 MB 47.0 MB/s 
     |████████████████████████████████| 596 kB 60.1 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

% cd /content/drive/MyDrive/dialog-generation-with-a-dynamic-human-preference-predictor/
% ls


Mounted at /content/drive
/content/drive/MyDrive/dialog-generation-with-a-dynamic-human-preference-predictor
data/                    get-pip.py        requirements.txt  tests/
dyme_models/             README.md         rl/
dyme_reward/             README_origin.md  seq2seq_models/
evaluation_and_results/  _report/          _slides/


In [4]:
import pandas as pd

In [5]:
results = pd.read_csv('generated_responses.csv')

In [6]:
results

,id,chat_history,gt_response,RLPA_response
0,0,[usr] Yeah about 10 years ago I had a horrifyi...,Did you suffer any injuries?,"Oh my gosh, I would have been so scared. What ..."
1,1,[usr] Yeah about 10 years ago I had a horrifyi...,Why did you feel guilty? People really shouldn...,What a relief that it turned out to be nothing...
2,2,"[usr] Well, can you tell me about your experie...",Yeah i wanted to tell you about the time i was...,I think you may be right. I can't remember the...
3,3,"[usr] Well, can you tell me about your experie...",Yeah he was punished hes in jail still,Yeah i am fine now but that was a long time ag...
4,4,"[usr] I there, dont know what to do, jst broke...",sorry to hear! do you have any idea about the ...,"Well, that's a long time. Maybe you should try..."
...,...,...,...,...
5250,5250,[usr] My friend showed me his Minecraft world ...,"That's super cool, I played a few years back b...",It sounds pretty cool. I'd like to see it some...
5251,5251,[usr] I had to cancel our family vacation comi...,I am really sorry to hear that. I hope everyt...,That's a bummer. Maybe you can take the kids t...
5252,5252,[usr] I had to cancel our family vacation comi...,"Well, that is just terrible. I hate it when c...",That is a real bummer. Maybe you guys can pla...
5253,5253,"[usr] Life is good, it's the gift that keeps o...",that is a great way to look at it,That's a good way of looking at it. It's good...


In [7]:
results['blenderbot_response'] = 0
results

,id,chat_history,gt_response,RLPA_response,blenderbot_response
0,0,[usr] Yeah about 10 years ago I had a horrifyi...,Did you suffer any injuries?,"Oh my gosh, I would have been so scared. What ...",0
1,1,[usr] Yeah about 10 years ago I had a horrifyi...,Why did you feel guilty? People really shouldn...,What a relief that it turned out to be nothing...,0
2,2,"[usr] Well, can you tell me about your experie...",Yeah i wanted to tell you about the time i was...,I think you may be right. I can't remember the...,0
3,3,"[usr] Well, can you tell me about your experie...",Yeah he was punished hes in jail still,Yeah i am fine now but that was a long time ag...,0
4,4,"[usr] I there, dont know what to do, jst broke...",sorry to hear! do you have any idea about the ...,"Well, that's a long time. Maybe you should try...",0
...,...,...,...,...,...
5250,5250,[usr] My friend showed me his Minecraft world ...,"That's super cool, I played a few years back b...",It sounds pretty cool. I'd like to see it some...,0
5251,5251,[usr] I had to cancel our family vacation comi...,I am really sorry to hear that. I hope everyt...,That's a bummer. Maybe you can take the kids t...,0
5252,5252,[usr] I had to cancel our family vacation comi...,"Well, that is just terrible. I hate it when c...",That is a real bummer. Maybe you guys can pla...,0
5253,5253,"[usr] Life is good, it's the gift that keeps o...",that is a great way to look at it,That's a good way of looking at it. It's good...,0


In [8]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [9]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Adapting/dialogue_agent_nlplab2022")

model = AutoModelForSeq2SeqLM.from_pretrained("Adapting/dialogue_agent_nlplab2022", revision = 'b86f62986872b4c1a9921acdb8cd226761d736cf').to(device)

In [22]:
tokenizer('[usr]')

{'input_ids': [8009, 2], 'attention_mask': [1, 1]}

In [27]:
def list_find(arr,ele):
    try:
        index = arr.index(ele)
    except:
        index = -1
    return index
    

In [38]:
def get_response(chat_history: str):
    input_ids = tokenizer(chat_history).input_ids
    if len(input_ids) > 128:
        input_ids = input_ids[-128:]
        index = list_find(input_ids,8009)
        if index != -1:
            input_ids = input_ids[index:]

    input_ids = torch.tensor(input_ids).view(1, -1).to(device)
    response = model.generate(input_ids)
    response = tokenizer.convert_ids_to_tokens(response[0],skip_special_tokens=True)
    response = tokenizer.convert_tokens_to_string(response).strip()
    return response

In [35]:
get_response('hello how are you?')

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 60 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


"I'm doing pretty good, how about yourself?  It's been a while since I've had a good night's sleep."

In [39]:
from tqdm import tqdm
for i in tqdm(range(len(results))):
    chat_history = results.loc[i,'chat_history']
    response = get_response(chat_history)
    results.loc[i,'blenderbot_response'] = response


  0%|          | 0/5255 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 60 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,
100%|██████████| 5255/5255 [44:52<00:00,  1.95it/s]


In [42]:
results.to_csv('generated_responses.csv')